In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
import re

# Read in detailed data about movies

In [2]:
df = pd.read_csv('/Users/aleksandra/ds/metis/project_luther/project_luther/rotten_tomatoes2.csv')

In [3]:
df.head(2)

,url,score_c,rating_c,n_reviews_c,fresh_c,rotten_c,score_u,rating_u,n_reviews_u,director,box_office,runtime,in_theatre,on_disc,audience,studio
0,https://www.rottentomatoes.com/m/resident_evil...,35,4.4/10,85,30,55,51%,5.9/10,"58,527",Paul W.S. Anderson,"$26,844,692",P106M,2017-01-26T16:00:00-08:00,2017-05-15T17:00:00-07:00,R (for sequences of violence throughout),Sony Pictures
1,https://www.rottentomatoes.com/m/passengers_2016,31,4.9/10,229,70,159,63%,4.7/10,"52,600",Morten Tyldum,"$100,014,092",P116M,2016-12-20T16:00:00-08:00,2017-03-13T17:00:00-07:00,"PG-13 (for sexuality, nudity and action/peril)",Columbia Pictures


What are the variables all about?

Variable | Explanation|
:---|:---
title | Title of the movie
url_uni | Url to the details of the movie
score_c | Percentage of Approved Tomatometer critics who gave a positive review to the movie
rating_c | Average rating according to the critics
n_reviews_c | Number of critics that have reviewed the movie
fresh_c | Number of positive reviews by critics
rotten_c | Number of negative reviews by critics
score_u | Percentage of the audience that gave positive review
rating_u | Average rating according to audience
director | Director
box_office | Box office
runtime | Runtime
in_theatre | Release date in theaters
on_disc | Release date on disc/streaming
audience | Rating and reason for rating
studio | Studio
audience_rating | Rating without explanation

In [5]:
len(df)

1484

In [6]:
df.dtypes

url            object
score_c         int64
rating_c       object
n_reviews_c     int64
fresh_c         int64
rotten_c        int64
score_u        object
rating_u       object
n_reviews_u    object
director       object
box_office     object
runtime        object
in_theatre     object
on_disc        object
audience       object
studio         object
dtype: object

# Convert data to right data type

Several columns are in the wrong dtype. Clean the data!  
For both types of ratings, some of them are from 0 to 10, some of them from 0 to 5. I will normalize the data so that they are all in the same unit.

In [7]:
def normalize_rating(series):
    splitted = series.str.split('/')
    normal = [float(row[0])/float(row[1]) for row in splitted]
    return normal

def to_numeric(series):
    return pd.to_numeric(series.replace(to_replace= '\D', value= '', regex=True))

In [8]:
df.rating_c = normalize_rating(df.rating_c)
df.rating_u = normalize_rating(df.rating_u)

In [14]:
df.score_u = to_numeric(df.score_u)
df.n_reviews_u = to_numeric(df.n_reviews_u)
df.box_office = to_numeric(df.box_office)
df.runtime = to_numeric(df.runtime)

In [18]:
df.in_theatre = pd.to_datetime(df.in_theatre)
df.on_disc = pd.to_datetime(df.on_disc)

In [20]:
df.head()

,url,score_c,rating_c,n_reviews_c,fresh_c,rotten_c,score_u,rating_u,n_reviews_u,director,box_office,runtime,in_theatre,on_disc,audience,studio
0,https://www.rottentomatoes.com/m/resident_evil...,35,0.44,85,30,55,51.0,0.59,58527,Paul W.S. Anderson,26844692.0,106.0,2017-01-27,2017-05-16,R (for sequences of violence throughout),Sony Pictures
1,https://www.rottentomatoes.com/m/passengers_2016,31,0.49,229,70,159,63.0,0.47,52600,Morten Tyldum,100014092.0,116.0,2016-12-21,2017-03-14,"PG-13 (for sexuality, nudity and action/peril)",Columbia Pictures
2,https://www.rottentomatoes.com/m/beauty_and_th...,32,0.46,19,6,13,53.0,0.66,3681,Christophe Gans,NaN,112.0,2014-01-03,2017-02-21,"PG-13 (for some action violence, peril and fri...",NaN
3,https://www.rottentomatoes.com/m/fantastic_bea...,73,0.68,271,198,73,79.0,0.70,84702,David Yates (II),234018657.0,132.0,2016-11-18,2017-03-28,PG-13 (for some fantasy action violence),Warner Bros. Pictures
4,https://www.rottentomatoes.com/m/absolutely_an...,18,0.37,40,7,33,31.0,0.54,3080,Terry Jones,673096.0,85.0,2017-05-12,2017-06-27,"NR (for language including sexual references, ...",Atlas Distribution


In [21]:
df.dtypes

url                    object
score_c                 int64
rating_c              float64
n_reviews_c             int64
fresh_c                 int64
rotten_c                int64
score_u               float64
rating_u              float64
n_reviews_u             int64
director               object
box_office            float64
runtime               float64
in_theatre     datetime64[ns]
on_disc        datetime64[ns]
audience               object
studio                 object
dtype: object

In [22]:
df['audience_rating'] = [e[0] for e in df.audience.str.split()]

In [23]:
df.head(2)

,url,score_c,rating_c,n_reviews_c,fresh_c,rotten_c,score_u,rating_u,n_reviews_u,director,box_office,runtime,in_theatre,on_disc,audience,studio,audience_rating
0,https://www.rottentomatoes.com/m/resident_evil...,35,0.44,85,30,55,51.0,0.59,58527,Paul W.S. Anderson,26844692.0,106.0,2017-01-27,2017-05-16,R (for sequences of violence throughout),Sony Pictures,R
1,https://www.rottentomatoes.com/m/passengers_2016,31,0.49,229,70,159,63.0,0.47,52600,Morten Tyldum,100014092.0,116.0,2016-12-21,2017-03-14,"PG-13 (for sexuality, nudity and action/peril)",Columbia Pictures,PG-13


In [24]:
df.shape

(1484, 17)

# Merge with title df

Merge this df with the df obtained with Selenium, containing urls and titles. Merge on url.

In [25]:
df_sel = pd.read_pickle('/Users/aleksandra/ds/metis/project_luther/all_data.pkl')

In [26]:
df_sel.head()

,title,url
0,Resident Evil: The Final Chapter,https://www.rottentomatoes.com/m/resident_evil...
1,Passengers,https://www.rottentomatoes.com/m/passengers_2016
2,Beauty And The Beast (La Belle Et La Bête),https://www.rottentomatoes.com/m/beauty_and_th...
3,Fantastic Beasts And Where To Find Them,https://www.rottentomatoes.com/m/fantastic_bea...
4,Absolutely Anything,https://www.rottentomatoes.com/m/absolutely_an...


In [27]:
len(df_sel)

1489

In [28]:
len(df)

1484

There is a difference in length, but this is expected: in the Scrapy log it was impossible to get 5 of the url's.

In [29]:
df_all_data = df_sel.merge(df, how = 'left', left_on = 'url', right_on='url')

In [30]:
len(df_all_data)

1489

In [31]:
len(df_all_data[df_all_data.rating_c.isnull()])

221

221 rows do not correspond in the dfs. I know that 5 movies were not scraped (log Scrapy). What happened with the remaining 216 movies?  

First step: identify the urls that were problematic, then inspect them to see why there is no correspondency.

In [32]:
common = df_sel.merge(df,on='url')
not_in_df = df_sel[(~df_sel.url.isin(common.url))]

In [33]:
not_in_df_sel = df[(~df.url.isin(common.url))]

In [34]:
urls_not_in_df = list(not_in_df.url.sort_values())

In [35]:
urls_not_in_df_sel = list(not_in_df_sel.url.sort_values())

In [37]:
print(urls_not_in_df[0])
print(urls_not_in_df_sel[0])

https://www.rottentomatoes.com/m/1000085-2001_a_space_odyssey
https://www.rottentomatoes.com/m/1000085_2001_a_space_odyssey


Some urls have '-' where they have '_' in the other df. We need to correct them...

In [38]:
df['url_uni'] = df.url.str.replace('-', '_')

In [39]:
df_sel['url_uni'] = df_sel.url.str.replace('-', '_')

In [40]:
df_all_data = df_sel.merge(df, on='url_uni')

In [41]:
len(df_all_data)

1484

That's better! Now we can drop the non-unified url columns.

In [42]:
df_all_data.head(2)

,title,url_x,url_uni,url_y,score_c,rating_c,n_reviews_c,fresh_c,rotten_c,score_u,rating_u,n_reviews_u,director,box_office,runtime,in_theatre,on_disc,audience,studio,audience_rating
0,Resident Evil: The Final Chapter,https://www.rottentomatoes.com/m/resident_evil...,https://www.rottentomatoes.com/m/resident_evil...,https://www.rottentomatoes.com/m/resident_evil...,35,0.44,85,30,55,51.0,0.59,58527,Paul W.S. Anderson,26844692.0,106.0,2017-01-27,2017-05-16,R (for sequences of violence throughout),Sony Pictures,R
1,Passengers,https://www.rottentomatoes.com/m/passengers_2016,https://www.rottentomatoes.com/m/passengers_2016,https://www.rottentomatoes.com/m/passengers_2016,31,0.49,229,70,159,63.0,0.47,52600,Morten Tyldum,100014092.0,116.0,2016-12-21,2017-03-14,"PG-13 (for sexuality, nudity and action/peril)",Columbia Pictures,PG-13


In [43]:
df_all_data.drop(['url_x', 'url_y'], axis=1, inplace=True)

In [44]:
df_all_data.head(2)

,title,url_uni,score_c,rating_c,n_reviews_c,fresh_c,rotten_c,score_u,rating_u,n_reviews_u,director,box_office,runtime,in_theatre,on_disc,audience,studio,audience_rating
0,Resident Evil: The Final Chapter,https://www.rottentomatoes.com/m/resident_evil...,35,0.44,85,30,55,51.0,0.59,58527,Paul W.S. Anderson,26844692.0,106.0,2017-01-27,2017-05-16,R (for sequences of violence throughout),Sony Pictures,R
1,Passengers,https://www.rottentomatoes.com/m/passengers_2016,31,0.49,229,70,159,63.0,0.47,52600,Morten Tyldum,100014092.0,116.0,2016-12-21,2017-03-14,"PG-13 (for sexuality, nudity and action/peril)",Columbia Pictures,PG-13


Pickle it!

In [46]:
df_all_data.to_pickle('df_all_data.pkl')

# Select variables relevant for analysis

The objective is to know which variables make a movie more or less popular. We want to take into account only those variables that can be chosen when a movie is made.

In [45]:
list(df_all_data)

['title',
 'url_uni',
 'score_c',
 'rating_c',
 'n_reviews_c',
 'fresh_c',
 'rotten_c',
 'score_u',
 'rating_u',
 'n_reviews_u',
 'director',
 'box_office',
 'runtime',
 'in_theatre',
 'on_disc',
 'audience',
 'studio',
 'audience_rating']

In [47]:
df_for_analysis = df_all_data.drop(['url_uni', 'n_reviews_c', 'fresh_c', 'rotten_c', 'n_reviews_u', 'box_office', 
                                    'on_disc', 'audience'], 1)

In [48]:
df_for_analysis.head()

,title,score_c,rating_c,score_u,rating_u,director,runtime,in_theatre,studio,audience_rating
0,Resident Evil: The Final Chapter,35,0.44,51.0,0.59,Paul W.S. Anderson,106.0,2017-01-27,Sony Pictures,R
1,Passengers,31,0.49,63.0,0.47,Morten Tyldum,116.0,2016-12-21,Columbia Pictures,PG-13
2,Beauty And The Beast (La Belle Et La Bête),32,0.46,53.0,0.66,Christophe Gans,112.0,2014-01-03,NaN,PG-13
3,Fantastic Beasts And Where To Find Them,73,0.68,79.0,0.70,David Yates (II),132.0,2016-11-18,Warner Bros. Pictures,PG-13
4,Absolutely Anything,18,0.37,31.0,0.54,Terry Jones,85.0,2017-05-12,Atlas Distribution,NR


The release date is not interesting in itself for the current question, but there might be months that are better for release than others, so I will create a new column 'month'.

In [49]:
df_for_analysis['month'] = df_for_analysis.in_theatre.dt.strftime('%b')

In [50]:
df_for_analysis.head()

,title,score_c,rating_c,score_u,rating_u,director,runtime,in_theatre,studio,audience_rating,month
0,Resident Evil: The Final Chapter,35,0.44,51.0,0.59,Paul W.S. Anderson,106.0,2017-01-27,Sony Pictures,R,Jan
1,Passengers,31,0.49,63.0,0.47,Morten Tyldum,116.0,2016-12-21,Columbia Pictures,PG-13,Dec
2,Beauty And The Beast (La Belle Et La Bête),32,0.46,53.0,0.66,Christophe Gans,112.0,2014-01-03,NaN,PG-13,Jan
3,Fantastic Beasts And Where To Find Them,73,0.68,79.0,0.70,David Yates (II),132.0,2016-11-18,Warner Bros. Pictures,PG-13,Nov
4,Absolutely Anything,18,0.37,31.0,0.54,Terry Jones,85.0,2017-05-12,Atlas Distribution,NR,May


In [51]:
df_for_analysis = df_for_analysis.drop('in_theatre', 1)

Pickle it!

In [52]:
df_for_analysis.to_pickle('df_for_analysis.pkl')